# 任务3 特征工程&特征选择(3天)

# 特征工程

In [1]:
#核心代码举例

# # 统计特征
#     #计算均值
#     gp = train.groupby(by)[fea].mean()
#     #计算中位数
#     gp = train.groupby(by)[fea].median()
#     #计算方差
#     gp = train.groupby(by)[fea].std()
#     #计算最大值
#     gp = train.groupby(by)[fea].max()
#     #计算最小值
#     gp = train.groupby(by)[fea].min()
#     #计算出现次数
#     gp = train.groupby(by)[fea].size()
    

# # groupby生成统计特征：mean,std
#     # 按照communityName分组计算面积的均值和方差
#     temp = data.groupby('communityName')['area'].agg({'com_area_mean': 'mean', 'com_area_std': 'std'})

# # 特征拆分
#     # 将houseType转为'Room'，'Hall'，'Bath'
#     def Room(x):
#         Room = int(x.split('室')[0])
#         return Room
#     def Hall(x):
#         Hall = int(x.split("室")[1].split("厅")[0])
#         return Hall
#     def Bath(x):
#         Bath = int(x.split("室")[1].split("厅")[1].split("卫")[0])
#         return Bath

#     data['Room'] = data['houseType'].apply(lambda x: Room(x))
#     data['Hall'] = data['houseType'].apply(lambda x: Hall(x))
#     data['Bath'] = data['houseType'].apply(lambda x: Bath(x))
    
# #特征合并
#     # 合并部分配套设施特征
#     data['trainsportNum'] = 5 * data['subwayStationNum'] / data['subwayStationNum'].mean() + data['busStationNum'] / \
#                                                                                              data[
#                                                                                                  'busStationNum'].mean()

# # 交叉生成特征:特征之间交叉+ - * / 
# data['Room_Bath'] = (data['Bath']+1) / (data['Room']+1)


# # 聚类特征
# from sklearn.mixture import GaussianMixture  使用GaussianMixture做聚类特征
# gmm = GaussianMixture(n_components=4, covariance_type='full', random_state=0)
# gmm.fit_predict(data)
 
# # 特征编码
# from sklearn.preprocessing import LabelEncoder
# data['communityName'] = LabelEncoder().fit_transform(data['communityName'])
# from sklearn import preprocessing.OneHotEncoder
# data['communityName'] = OneHotEncoder().fit_transform(data['communityName'])


# # 过大量级值取log平滑（针对线性模型有效）
# data[feature]=np.log1p(data[feature])

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv('./数据集/train_data.csv')
test = pd.read_csv('./数据集/test_a.csv')
# print(test.columns.to_list())
# print(train.shape,test.shape)

# target_train = train['tradeMoney']
target_test = pd.read_csv('./数据集/评分文件/sub_a_913.csv')
# print(target_test)
# print(target_train.head())
print(train.shape,test.shape)
# target_test = test.pop('tradeMoney')


(41440, 51) (2469, 50)


## 预处理和lableEncode()

In [3]:
def preprocessingData(data):
    # 填充缺失值
    data['rentType'][data['rentType'] == '--'] = '未知方式'
    
    # 转换object类型数据
    columns = ['rentType','communityName', 'houseFloor', 'houseToward', 'houseDecoration',  'region', 'plate'] #,'houseType'
    
    for feature in columns:
        data[feature] = LabelEncoder().fit_transform(data[feature])

    # 将buildYear列转换为整型数据
    buildYearmean = pd.DataFrame(data[data['buildYear'] != '暂无信息']['buildYear'].mode())
    data.loc[data[data['buildYear'] == '暂无信息'].index, 'buildYear'] = buildYearmean.iloc[0, 0]
    data['buildYear'] = data['buildYear'].astype('int')

    # 处理pv和uv的空值
    data['pv'].fillna(data['pv'].mean(), inplace=True)
    data['uv'].fillna(data['uv'].mean(), inplace=True)
    data['pv'] = data['pv'].astype('int')
    data['uv'] = data['uv'].astype('int')

    # 分割交易时间
    def month(x):
        month = int(x.split('/')[1])
        return month
    def day(x):
        day = int(x.split('/')[2])
        return day
    data['month'] = data['tradeTime'].apply(lambda x: month(x))
#     data['day'] = data['tradeTime'].apply(lambda x: day(x))
    
#     # 去掉部分特征
    data.drop('city', axis=1, inplace=True)
    data.drop('tradeTime', axis=1, inplace=True)
    data.drop('ID', axis=1, inplace=True)
    return data

train = preprocessingData(train)
test = preprocessingData(test)
# print(data_train['rentType'])
# sns.distplot([data_train['rentType']])
# plt.hist(data_train['rentType'].value_counts())

## 2-异常值处理

In [4]:
# clean data
from sklearn.ensemble import IsolationForest
def IF_drop(train):
    IForest = IsolationForest(contamination=0.01)
    IForest.fit(train["tradeMoney"].values.reshape(-1,1))
    y_pred = IForest.predict(train["tradeMoney"].values.reshape(-1,1))
    drop_index = train.loc[y_pred==-1].index
    print(drop_index)
    train.drop(drop_index,inplace=True)
    return train

train = IF_drop(train)

d:\python\lib\site-packages\sklearn\ensemble\iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


Int64Index([   62,    69,   128,   131,   246,   261,   266,   297,   308,
              313,
            ...
            39224, 39228, 39319, 39347, 39352, 39434, 39563, 41080, 41083,
            41233],
           dtype='int64', length=405)


In [5]:
def dropData(train):
    # 丢弃部分异常值
    train = train[train.area <= 200]
    train = train[(train.tradeMoney <=16000) & (train.tradeMoney >=700)]
    train.drop(train[(train['totalFloor'] == 0)].index, inplace=True)
    return train  
#数据集异常值处理
train = dropData(train)

## 2- 深度清理

In [6]:
def cleanData(data):
    data.drop(data[(data['region']=='RG00001') & (data['tradeMoney']<1000)&(data['area']>50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00001') & (data['tradeMoney']>25000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00001') & (data['area']>250)&(data['tradeMoney']<20000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00001') & (data['area']>400)&(data['tradeMoney']>50000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00001') & (data['area']>100)&(data['tradeMoney']<2000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00002') & (data['area']<100)&(data['tradeMoney']>60000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003') & (data['area']<300)&(data['tradeMoney']>30000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003') & (data['tradeMoney']<500)&(data['area']<50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003') & (data['tradeMoney']<1500)&(data['area']>100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003') & (data['tradeMoney']<2000)&(data['area']>300)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003') & (data['tradeMoney']>5000)&(data['area']<20)].index,inplace=True)
    data.drop(data[(data['region']=='RG00003') & (data['area']>600)&(data['tradeMoney']>40000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00004') & (data['tradeMoney']<1000)&(data['area']>80)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006') & (data['tradeMoney']<200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005') & (data['tradeMoney']<2000)&(data['area']>180)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005') & (data['tradeMoney']>50000)&(data['area']<200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006') & (data['area']>200)&(data['tradeMoney']<2000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00007') & (data['area']>100)&(data['tradeMoney']<2500)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010') & (data['area']>200)&(data['tradeMoney']>25000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010') & (data['area']>400)&(data['tradeMoney']<15000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010') & (data['tradeMoney']<3000)&(data['area']>200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010') & (data['tradeMoney']>7000)&(data['area']<75)].index,inplace=True)
    data.drop(data[(data['region']=='RG00010') & (data['tradeMoney']>12500)&(data['area']<100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00004') & (data['area']>400)&(data['tradeMoney']>20000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00008') & (data['tradeMoney']<2000)&(data['area']>80)].index,inplace=True)
    data.drop(data[(data['region']=='RG00009') & (data['tradeMoney']>40000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00009') & (data['area']>300)].index,inplace=True)
    data.drop(data[(data['region']=='RG00009') & (data['area']>100)&(data['tradeMoney']<2000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00011') & (data['tradeMoney']<10000)&(data['area']>390)].index,inplace=True)
    data.drop(data[(data['region']=='RG00012') & (data['area']>120)&(data['tradeMoney']<5000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00013') & (data['area']<100)&(data['tradeMoney']>40000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00013') & (data['area']>400)&(data['tradeMoney']>50000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00013') & (data['area']>80)&(data['tradeMoney']<2000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014') & (data['area']>300)&(data['tradeMoney']>40000)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014') & (data['tradeMoney']<1300)&(data['area']>80)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014') & (data['tradeMoney']<8000)&(data['area']>200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014') & (data['tradeMoney']<1000)&(data['area']>20)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014') & (data['tradeMoney']>25000)&(data['area']>200)].index,inplace=True)
    data.drop(data[(data['region']=='RG00014') & (data['tradeMoney']<20000)&(data['area']>250)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005') & (data['tradeMoney']>30000)&(data['area']<100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005') & (data['tradeMoney']<50000)&(data['area']>600)].index,inplace=True)
    data.drop(data[(data['region']=='RG00005') & (data['tradeMoney']>50000)&(data['area']>350)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006') & (data['tradeMoney']>4000)&(data['area']<100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006') & (data['tradeMoney']<600)&(data['area']>100)].index,inplace=True)
    data.drop(data[(data['region']=='RG00006') & (data['area']>165)].index,inplace=True)
    data.drop(data[(data['region']=='RG00012') & (data['tradeMoney']<800)&(data['area']<30)].index,inplace=True)
    data.drop(data[(data['region']=='RG00007') & (data['tradeMoney']<1100)&(data['area']>50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00004') & (data['tradeMoney']>8000)&(data['area']<80)].index,inplace=True)
    data.loc[(data['region']=='RG00002')&(data['area']>50)&(data['rentType']=='合租'),'rentType']='整租'
    data.loc[(data['region']=='RG00014')&(data['rentType']=='合租')&(data['area']>60),'rentType']='整租'
    data.drop(data[(data['region']=='RG00008')&(data['tradeMoney']>15000)&(data['area']<110)].index,inplace=True)
    data.drop(data[(data['region']=='RG00008')&(data['tradeMoney']>20000)&(data['area']>110)].index,inplace=True)
    data.drop(data[(data['region']=='RG00008')&(data['tradeMoney']<1500)&(data['area']<50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00008')&(data['rentType']=='合租')&(data['area']>50)].index,inplace=True)
    data.drop(data[(data['region']=='RG00015') ].index,inplace=True)
    data.reset_index(drop=True, inplace=True)
    return data

train = cleanData(train)
target_train = train.pop('tradeMoney')

## 特征合并

In [7]:
def newfeature(data):


    # 将houseType转为'Room'，'Hall'，'Bath'
    def Room(x):
        Room = int(x.split('室')[0])
        return Room
    def Hall(x):
        Hall = int(x.split("室")[1].split("厅")[0])
        return Hall
    def Bath(x):
        Bath = int(x.split("室")[1].split("厅")[1].split("卫")[0])
        return Bath

    data['Room'] = data['houseType'].apply(lambda x: Room(x))
    data['Hall'] = data['houseType'].apply(lambda x: Hall(x))
    data['Bath'] = data['houseType'].apply(lambda x: Bath(x))
    data['Room_Bath'] = (data['Bath']+1) / (data['Room']+1)
    # 填充租房类型
    data.loc[(data['rentType'] == '未知方式') & (data['Room'] <= 1), 'rentType'] = '整租'
    # print(data.loc[(data['rentType']=='未知方式')&(data['Room_Bath']>1),'rentType'])
    data.loc[(data['rentType'] == '未知方式') & (data['Room_Bath'] > 1), 'rentType'] = '合租'
    data.loc[(data['rentType'] == '未知方式') & (data['Room'] > 1) & (data['area'] < 50), 'rentType'] = '合租'
    data.loc[(data['rentType'] == '未知方式') & (data['area'] / data['Room'] < 20), 'rentType'] = '合租'
    # data.loc[(data['rentType']=='未知方式')&(data['area']>60),'rentType']='合租'
    data.loc[(data['rentType'] == '未知方式') & (data['area'] <= 50) & (data['Room'] == 2), 'rentType'] = '合租'
    data.loc[(data['rentType'] == '未知方式') & (data['area'] > 60) & (data['Room'] == 2), 'rentType'] = '整租'
    data.loc[(data['rentType'] == '未知方式') & (data['area'] <= 60) & (data['Room'] == 3), 'rentType'] = '合租'
    data.loc[(data['rentType'] == '未知方式') & (data['area'] > 60) & (data['Room'] == 3), 'rentType'] = '整租'
    data.loc[(data['rentType'] == '未知方式') & (data['area'] >= 100) & (data['Room'] > 3), 'rentType'] = '整租'

    # data.drop('Room_Bath', axis=1, inplace=True)
    # 提升0.0001
#     def month(x):
#         month = int(x.split('/')[1])
#         return month
#     def day(x):
#         day = int(x.split('/')[2])
#         return day
    # 结果变差

    # 分割交易时间
#     data['year']=data['tradeTime'].apply(lambda x:year(x))
#     data['month'] = data['tradeTime'].apply(lambda x: month(x))
#     data['day'] = data['tradeTime'].apply(lambda x: day(x))# 结果变差
    #     data['pv/uv'] = data['pv'] / data['uv']
    #     data['房间总数'] = data['室'] + data['厅'] + data['卫']

    # 合并部分配套设施特征
    data['trainsportNum'] = 5 * data['subwayStationNum'] / data['subwayStationNum'].mean() + data['busStationNum'] / \
                                                                                             data[
                                                                                                 'busStationNum'].mean()
    data['all_SchoolNum'] = 2 * data['interSchoolNum'] / data['interSchoolNum'].mean() + data['schoolNum'] / data[
        'schoolNum'].mean() \
                            + data['privateSchoolNum'] / data['privateSchoolNum'].mean()
    data['all_hospitalNum'] = 2 * data['hospitalNum'] / data['hospitalNum'].mean() + \
                              data['drugStoreNum'] / data['drugStoreNum'].mean()
    data['all_mall'] = data['mallNum'] / data['mallNum'].mean() + \
                       data['superMarketNum'] / data['superMarketNum'].mean()
    data['otherNum'] = data['gymNum'] / data['gymNum'].mean() + data['bankNum'] / data['bankNum'].mean() + \
                       data['shopNum'] / data['shopNum'].mean() + 2 * data['parkNum'] / data['parkNum'].mean()

    data.drop(['subwayStationNum', 'busStationNum',
               'interSchoolNum', 'schoolNum', 'privateSchoolNum',
               'hospitalNum', 'drugStoreNum', 'mallNum', 'superMarketNum', 'gymNum', 'bankNum', 'shopNum', 'parkNum'],
              axis=1, inplace=True)
    # 提升0.0005
    
#     data['houseType_1sumcsu']=data['Bath'].map(lambda x:str(x))+data['month'].map(lambda x:str(x))
#     data['houseType_2sumcsu']=data['Bath'].map(lambda x:str(x))+data['communityName']
#     data['houseType_3sumcsu']=data['Bath'].map(lambda x:str(x))+data['plate']
    
    data.drop('houseType', axis=1, inplace=True)
#     data.drop('tradeTime', axis=1, inplace=True)
    
    data["area"] = data["area"].astype(int)


    # categorical_feats = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration', 'communityName','region', 'plate']
    categorical_feats = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration',  'region', 'plate','cluster']

    return data, categorical_feats

train,_ = newfeature(train)
test,_ = newfeature(test)


## 计算统计特征

In [8]:
#计算统计特征
def featureCount(train,test):
    train['data_type'] = 0
    test['data_type'] = 1
    data = pd.concat([train, test], axis=0, join='outer')
    def feature_count(data, features=[]):
        new_feature = 'count'
        for i in features:
            new_feature += '_' + i
        temp = data.groupby(features).size().reset_index().rename(columns={0: new_feature})
        data = data.merge(temp, 'left', on=features)
        return data

    data = feature_count(data, ['communityName'])
    data = feature_count(data, ['buildYear'])
    data = feature_count(data, ['totalFloor'])
    data = feature_count(data, ['communityName', 'totalFloor'])
    data = feature_count(data, ['communityName', 'newWorkers'])
    data = feature_count(data, ['communityName', 'totalTradeMoney'])
    new_train = data[data['data_type'] == 0]
    new_test = data[data['data_type'] == 1]
    new_train.drop('data_type', axis=1, inplace=True)
    new_test.drop(['data_type'], axis=1, inplace=True)
    return new_train, new_test
    
train, test = featureCount(train, test)

## groupby方法生成统计特征

In [9]:
#groupby生成统计特征：mean,std等

def gourpby(train,test):
    train['data_type'] = 0
    test['data_type'] = 1
    data = pd.concat([train, test], axis=0, join='outer')
    columns = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration', 'communityName', 'region', 'plate']
    for feature in columns:
        data[feature] = LabelEncoder().fit_transform(data[feature])

    temp = data.groupby('communityName')['area'].agg({'com_area_mean': 'mean', 'com_area_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='communityName', how='left')
    
    data['price_per_area'] = data.tradeMeanPrice / data.area * 100
    temp = data.groupby('communityName')['price_per_area'].agg(
        {'comm_price_mean': 'mean', 'comm_price_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='communityName', how='left')
   
    temp = data.groupby('plate')['price_per_area'].agg(
        {'plate_price_mean': 'mean', 'plate_price_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='plate', how='left')
    data.drop('price_per_area', axis=1, inplace=True)

    temp = data.groupby('plate')['area'].agg({'plate_area_mean': 'mean', 'plate_area_std': 'std'})
    temp.fillna(0, inplace=True)
    data = data.merge(temp, on='plate', how='left')
    
    temp = data.groupby(['plate'])['buildYear'].agg({'plate_year_mean': 'mean', 'plate_year_std': 'std'})
    data = data.merge(temp, on='plate', how='left')
    data.plate_year_mean = data.plate_year_mean.astype('int')
    data['comm_plate_year_diff'] = data.buildYear - data.plate_year_mean
    data.drop('plate_year_mean', axis=1, inplace=True)

    temp = data.groupby('plate')['trainsportNum'].agg('sum').reset_index(name='plate_trainsportNum')
    data = data.merge(temp, on='plate', how='left')
    temp = data.groupby(['communityName', 'plate'])['trainsportNum'].agg('sum').reset_index(name='com_trainsportNum')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')
    data['trainsportNum_ratio'] = list(map(lambda x, y: round(x / y, 3) if y != 0 else -1,
                                           data['com_trainsportNum'], data['plate_trainsportNum']))
    data = data.drop(['com_trainsportNum', 'plate_trainsportNum'], axis=1)

    temp = data.groupby('plate')['all_SchoolNum'].agg('sum').reset_index(name='plate_all_SchoolNum')
    data = data.merge(temp, on='plate', how='left')
    temp = data.groupby(['communityName', 'plate'])['all_SchoolNum'].agg('sum').reset_index(name='com_all_SchoolNum')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')
    data = data.drop(['com_all_SchoolNum', 'plate_all_SchoolNum'], axis=1)

    temp = data.groupby(['communityName', 'plate'])['all_mall'].agg('sum').reset_index(name='com_all_mall')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')

    temp = data.groupby('plate')['otherNum'].agg('sum').reset_index(name='plate_otherNum')
    data = data.merge(temp, on='plate', how='left')
    temp = data.groupby(['communityName', 'plate'])['otherNum'].agg('sum').reset_index(name='com_otherNum')
    data = data.merge(temp, on=['communityName', 'plate'], how='left')
    data['other_ratio'] = list(map(lambda x, y: round(x / y, 3) if y != 0 else -1,
                                   data['com_otherNum'], data['plate_otherNum']))
    data = data.drop(['com_otherNum', 'plate_otherNum'], axis=1)

    temp = data.groupby(['month', 'communityName']).size().reset_index(name='communityName_saleNum')
    data = data.merge(temp, on=['month', 'communityName'], how='left')
    temp = data.groupby(['month', 'plate']).size().reset_index(name='plate_saleNum')
    data = data.merge(temp, on=['month', 'plate'], how='left')

    data['sale_ratio'] = round((data.communityName_saleNum + 1) / (data.plate_saleNum + 1), 3)
    data['sale_newworker_differ'] = 3 * data.plate_saleNum - data.newWorkers
    data.drop(['communityName_saleNum', 'plate_saleNum'], axis=1, inplace=True)

    new_train = data[data['data_type'] == 0]
    new_test = data[data['data_type'] == 1]
    new_train.drop('data_type', axis=1, inplace=True)
    new_test.drop(['data_type'], axis=1, inplace=True)
    return new_train, new_test

train, test = gourpby(train, test)

## 聚类方法

In [10]:
#聚类
def cluster(train,test):
    from sklearn.mixture import GaussianMixture

    train['data_type'] = 0
    test['data_type'] = 1
    data = pd.concat([train, test], axis=0, join='outer')
    col = ['totalFloor',
           'houseDecoration', 'communityName', 'region', 'plate', 'buildYear',

           'tradeMeanPrice', 'tradeSecNum', 'totalNewTradeMoney',
           'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum',

           'landTotalPrice', 'landMeanPrice', 'totalWorkers',
           'newWorkers', 'residentPopulation', 'lookNum',
           'trainsportNum',
           'all_SchoolNum', 'all_hospitalNum', 'all_mall', 'otherNum']

    # EM
    gmm = GaussianMixture(n_components=3, covariance_type='full', random_state=0)
    data['cluster']= pd.DataFrame(gmm.fit_predict(data[col]))


    col1 = ['totalFloor','houseDecoration', 'communityName', 'region', 'plate', 'buildYear']
    col2 = ['tradeMeanPrice', 'tradeSecNum', 'totalNewTradeMoney',
            'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum',
            'landTotalPrice', 'landMeanPrice', 'totalWorkers',
            'newWorkers', 'residentPopulation', 'lookNum',
            'trainsportNum',
            'all_SchoolNum', 'all_hospitalNum', 'all_mall', 'otherNum']
    for feature1 in col1:
        for feature2 in col2:
        
            temp = data.groupby(['cluster',feature1])[feature2].agg('mean').reset_index(name=feature2+'_'+feature1+'_cluster_mean')
            temp.fillna(0, inplace=True)
       
            data = data.merge(temp, on=['cluster', feature1], how='left')
    
   
    new_train = data[data['data_type'] == 0]
    new_test = data[data['data_type'] == 1]
    new_train.drop('data_type', axis=1, inplace=True)
    new_test.drop(['data_type'], axis=1, inplace=True)
    
    return new_train, new_test

train, test = cluster(train, test)   

## log平滑

In [11]:
# 过大量级值取log平滑（针对线性模型有效）
big_num_cols = ['totalTradeMoney','totalTradeArea','tradeMeanPrice','totalNewTradeMoney', 'totalNewTradeArea',
                'tradeNewMeanPrice','remainNewNum', 'supplyNewNum', 'supplyLandArea',
                'tradeLandArea','landTotalPrice','landMeanPrice','totalWorkers','newWorkers',
                'residentPopulation','pv','uv']
for col in big_num_cols:
        train[col] = train[col].map(lambda x: np.log1p(x))
        test[col] = test[col].map(lambda x: np.log1p(x))

        
print(target_train.head())
print(train.shape,test.shape)
        

0     2000.0
1     2000.0
2    16000.0
3     1600.0
4     2900.0
Name: tradeMoney, dtype: float64
(40215, 173) (2469, 173)


In [12]:
print(target_train.shape)
# print(target_train)
print(train.head())
print(test.head())

(40215,)
   area  rentType  houseFloor  totalFloor  houseToward  houseDecoration  \
0    68         2           1          16            6                0   
1   125         2           0          14            6                2   
2   132         2           1          32            6                0   
3    57         2           0          17            6                3   
4   129         2           1           2            6                1   

   communityName  region  plate  buildYear  ...  \
0             50       0     63       1953  ...   
1            129       1     48       2007  ...   
2            178       1     49       1994  ...   
3            312       1     50       1994  ...   
4           1256       2     43       1994  ...   

   landMeanPrice_buildYear_cluster_mean  totalWorkers_buildYear_cluster_mean  \
0                              0.000000                        193780.461538   
1                            795.452036                         48519.514

In [13]:
#对比特征工程前后线性模型结果情况
test=test.fillna(0)
# Lasso回归
from sklearn.linear_model import Lasso
lasso=Lasso(alpha=0.1)
lasso.fit(train,target_train)
#预测测试集和训练集结果
y_pred_train=lasso.predict(train)
y_pred_test=lasso.predict(test)

#对比结果
from sklearn.metrics import r2_score
score_train=r2_score(y_pred_train,target_train)
print("训练集结果：",score_train)
score_test=r2_score(y_pred_test, target_test)
print("测试集结果：",score_test)

训练集结果： 0.7270259992174979
测试集结果： 0.306681676974662


# 模型选择

   机器学习模型的调参。有两种常用的调参方法：网格搜索和随机搜索。每一种都有自己的优点和缺点。网格搜索速度慢，但在搜索整个搜索空间方面效果很好，而随机搜索很快，但可能会错过搜索空间中的重要点。幸运的是，还有第三种选择：贝叶斯优化。本文我们将重点介绍贝叶斯优化的一个实现，一个名为hyperopt的 Python 模块。
    本文是对Parameter Tuning with Hyperopt一文的翻译。译者在设计深度学习模型的网络结构发现了hyperopt这个大杀器，相比每次手动各种试，用工具批量调节网络中的各种超参数确实能省心不少。不过hyperopt的官方文档描述的太渣，google 了一翻，发现这篇博客算是介绍的比较清楚的一个，便顺手翻译了，译文已取得原作者授权。

hyperopt详细中文解释：https://www.jianshu.com/p/35eed1567463

   模型选择的标准是尽可能地贴近样本真实的分布。但是在有限的样本下，如果我们有多个可选模型，比如从简单到复杂，从低阶到高阶，参数由少到多。那么我们怎么选择模型呢，是对训练样本的拟合度越好就可以吗？显然不是，因为这样做的话只会让我们最终选择出最复杂，最高阶的模型。而这个模型的问题是过拟合的，即对样本真实分布的预测误差是很高的。那么该如何选择模型，使得泛化误差尽量小呢，有下面这些常用的方法：
### 保留交叉验证        
把样本分成训练样本和测试样本，一般可以7比3的比例。7成的样本训练出的模型，用3成的样本做检验。取测试准确率最高的模型
###  K折交叉验证
取K为10为例，把所有样本平均分成10分，然后用9份训练，剩下的1份做测试。这样可以做十次测试，取十次测试的准确率的平均值最高的模型做为选取的模型。
###  留1交叉验证
当样本数目很少并且很难取得的时候，K折交叉验证的极限就是让K等于样本数目N，这样N-1个样本作为训练样本，1个作为测试样本。经过N次测试，取平局准确率最高的模型作为我们选择的模型。
在交叉验证做完选取理想模型之后，可以把所有样本再放到模型中训练一次，作为最后的输出模型。

In [14]:
X_train = train
Y_train = target_train

In [15]:
from __future__ import print_function
import lightgbm as lgb
import sklearn
import numpy
import hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import colorama
import numpy as np
from sklearn.model_selection import KFold

N_HYPEROPT_PROBES = 500
HYPEROPT_ALGO = tpe.suggest  #  tpe.suggest OR hyperopt.rand.suggest

# ----------------------------------------------------------

colorama.init()

# ---------------------------------------------------------------------

def get_lgb_params(space):
    lgb_params = dict()
    lgb_params['boosting_type'] = space['boosting_type'] if 'boosting_type' in space else 'gbdt'
    lgb_params['objective'] = 'regression'
    lgb_params['metric'] = 'rmse'
    lgb_params['learning_rate'] = space['learning_rate']
    lgb_params['num_leaves'] = int(space['num_leaves'])
    lgb_params['min_data_in_leaf'] = int(space['min_data_in_leaf'])
    lgb_params['min_sum_hessian_in_leaf'] = space['min_sum_hessian_in_leaf']
    lgb_params['max_depth'] = -1
    lgb_params['lambda_l1'] = space['lambda_l1'] if 'lambda_l1' in space else 0.0
    lgb_params['lambda_l2'] = space['lambda_l2'] if 'lambda_l2' in space else 0.0
    lgb_params['max_bin'] = int(space['max_bin']) if 'max_bin' in space else 256
    lgb_params['feature_fraction'] = space['feature_fraction']
    lgb_params['bagging_fraction'] = space['bagging_fraction']
    lgb_params['bagging_freq'] = int(space['bagging_freq']) if 'bagging_freq' in space else 1
    lgb_params['nthread'] = 4
    return lgb_params

# ---------------------------------------------------------------------

obj_call_count = 0
cur_best_score = 0 # 0 or np.inf
# log_writer = open( '../log/lgb-hyperopt-log.txt', 'w' )


def objective(space):
    global obj_call_count, cur_best_score

    obj_call_count += 1

    print('\nLightGBM objective call #{} cur_best_score={:7.5f}'.format(obj_call_count,cur_best_score) )

    lgb_params = get_lgb_params(space)

    sorted_params = sorted(space.items(), key=lambda z: z[0])
    params_str = str.join(' ', ['{}={}'.format(k, v) for k, v in sorted_params])
    print('Params: {}'.format(params_str) )
    
    kf = KFold(n_splits=3, shuffle=True, random_state=0)
    out_of_fold = np.zeros(len(X_train))
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        D_train = lgb.Dataset(X_train.iloc[train_idx], label=Y_train[train_idx])
        D_val = lgb.Dataset(X_train.iloc[val_idx], label=Y_train[val_idx])
        # Train
        num_round = 10000
        clf = lgb.train(lgb_params,
                           D_train,
                           num_boost_round=num_round,
                           # metrics='mlogloss',
                           valid_sets=D_val,
                           # valid_names='val',
                           # fobj=None,
                           # feval=None,
                           # init_model=None,
                           # feature_name='auto',
                           # categorical_feature='auto',
                           early_stopping_rounds=200,
                           # evals_result=None,
                           verbose_eval=False,
                           # learning_rates=None,
                           # keep_training_booster=False,
                           # callbacks=None
                           )
        # predict
        nb_trees = clf.best_iteration
        val_loss = clf.best_score['valid_0']
        print('nb_trees={} val_loss={}'.format(nb_trees, val_loss))
        out_of_fold[val_idx] = clf.predict(X_train.iloc[val_idx], num_iteration=nb_trees)
        score = r2_score(out_of_fold, Y_train)

    print('val_r2_score={}'.format(score))

#     log_writer.write('score={} Params:{} nb_trees={}\n'.format(score, params_str, nb_trees ))
#     log_writer.flush()

    if score>cur_best_score:
        cur_best_score = score
        print(colorama.Fore.GREEN + 'NEW BEST SCORE={}'.format(cur_best_score) + colorama.Fore.RESET)
    return {'loss': -score, 'status': STATUS_OK}

# --------------------------------------------------------------------------------

space ={
        'num_leaves': hp.quniform ('num_leaves', 10, 100, 1),
        'min_data_in_leaf':  hp.quniform ('min_data_in_leaf', 10, 100, 1),
        'feature_fraction': hp.uniform('feature_fraction', 0.75, 1.0),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.75, 1.0),
        'learning_rate': hp.uniform('learning_rate', 0, 0.01),
#         'learning_rate': hp.loguniform('learning_rate', -5.0, -2.3),
        'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', 0, 2.3),
        'max_bin': hp.quniform ('max_bin', 88, 200, 1),
        'bagging_freq': hp.quniform ('bagging_freq', 1, 15, 1),
        'lambda_l1': hp.uniform('lambda_l1', 0, 10 ),
        'lambda_l2': hp.uniform('lambda_l2', 0, 10 ),
       }

trials = Trials()
best = hyperopt.fmin(fn=objective,
                     space=space,
                     algo=HYPEROPT_ALGO,
                     max_evals=N_HYPEROPT_PROBES,
                     trials=trials,
                     verbose=1)

print('-'*50)
print('The best params:')
print( best )
print('\n\n')

                                                                                                                       
LightGBM objective call #1 cur_best_score=0.00000
Params: bagging_fraction=0.8401944845661764 bagging_freq=9.0 feature_fraction=0.9004764865751869 lambda_l1=2.2795304243957815 lambda_l2=0.6854493989713117 learning_rate=0.005817151174194069 max_bin=192.0 min_data_in_leaf=40.0 min_sum_hessian_in_leaf=2.0422427668137093 num_leaves=44.0
nb_trees=5512 val_loss=OrderedDict([('rmse', 792.9034534207018)])                                                      
nb_trees=6443 val_loss=OrderedDict([('rmse', 803.0143027271312)])                                                      
nb_trees=4770 val_loss=OrderedDict([('rmse', 763.1559140197394)])                                                      
val_r2_score=0.8898141919430032                                                                                        
NEW BEST SCORE=0.8898141919430032                                

nb_trees=4729 val_loss=OrderedDict([('rmse', 800.1255766919248)])                                                      
nb_trees=4209 val_loss=OrderedDict([('rmse', 763.8783017327704)])                                                      
val_r2_score=0.8906014208728932                                                                                        
NEW BEST SCORE=0.8906014208728932                                                                                      
                                                                                                                       
LightGBM objective call #10 cur_best_score=0.89060
Params: bagging_fraction=0.8909925103308124 bagging_freq=7.0 feature_fraction=0.8274571591885987 lambda_l1=0.6638241105756837 lambda_l2=9.963672292762567 learning_rate=0.0024982133311787845 max_bin=168.0 min_data_in_leaf=63.0 min_sum_hessian_in_leaf=1.079989325520889 num_leaves=30.0
nb_trees=9988 val_loss=OrderedDict([('rmse', 800.0015253901739)])

Params: bagging_fraction=0.8320138062995025 bagging_freq=8.0 feature_fraction=0.8951031969314941 lambda_l1=4.236261515208931 lambda_l2=5.8376562889541725 learning_rate=0.0007437942174573597 max_bin=196.0 min_data_in_leaf=27.0 min_sum_hessian_in_leaf=4.816264983101335 num_leaves=83.0
nb_trees=10000 val_loss=OrderedDict([('rmse', 807.9381087614074)])                                                     
nb_trees=10000 val_loss=OrderedDict([('rmse', 825.7432488810391)])                                                     
nb_trees=10000 val_loss=OrderedDict([('rmse', 782.1622312694922)])                                                     
val_r2_score=0.8811854034421074                                                                                        
                                                                                                                       
LightGBM objective call #19 cur_best_score=0.89060
Params: bagging_fraction=0.7990368179779607 bagging_freq=4.0 feat

val_r2_score=0.8859662298968455                                                                                        
                                                                                                                       
LightGBM objective call #27 cur_best_score=0.89060
Params: bagging_fraction=0.8873036506563471 bagging_freq=2.0 feature_fraction=0.7552045900875545 lambda_l1=8.83728186603232 lambda_l2=5.519022129230761 learning_rate=0.005013342217530346 max_bin=183.0 min_data_in_leaf=32.0 min_sum_hessian_in_leaf=9.903694446756495 num_leaves=71.0
nb_trees=5451 val_loss=OrderedDict([('rmse', 788.815502798187)])                                                       
nb_trees=4682 val_loss=OrderedDict([('rmse', 800.2932256698994)])                                                      
nb_trees=4426 val_loss=OrderedDict([('rmse', 763.4845282029127)])                                                      
val_r2_score=0.8901759286437191                                     

nb_trees=3666 val_loss=OrderedDict([('rmse', 801.481587856809)])                                                       
nb_trees=2980 val_loss=OrderedDict([('rmse', 762.7186510842719)])                                                      
val_r2_score=0.8899662742441973                                                                                        
                                                                                                                       
LightGBM objective call #36 cur_best_score=0.89060
Params: bagging_fraction=0.9226557899148194 bagging_freq=7.0 feature_fraction=0.950393895051474 lambda_l1=2.152925600057301 lambda_l2=6.293750457679054 learning_rate=0.005898196300656578 max_bin=148.0 min_data_in_leaf=43.0 min_sum_hessian_in_leaf=8.60072927096563 num_leaves=22.0
nb_trees=9988 val_loss=OrderedDict([('rmse', 795.4592928562585)])                                                      
nb_trees=8981 val_loss=OrderedDict([('rmse', 806.5503780100858)])    

...

nb_trees=2952 val_loss=OrderedDict([('rmse', 805.9214025432639)])                                                      
nb_trees=2395 val_loss=OrderedDict([('rmse', 764.4925974348791)])                                                      
val_r2_score=0.8895871797714049                                                                                        
                                                                                                                       
LightGBM objective call #140 cur_best_score=0.89078
Params: bagging_fraction=0.9241725079938402 bagging_freq=6.0 feature_fraction=0.9210403940285158 lambda_l1=8.446358096877619 lambda_l2=6.739881079074073 learning_rate=0.00013893130589839962 max_bin=129.0 min_data_in_leaf=29.0 min_sum_hessian_in_leaf=7.158163475173604 num_leaves=36.0
nb_trees=10000 val_loss=OrderedDict([('rmse', 1218.7050790378898)])                                                    
nb_trees=10000 val_loss=OrderedDict([('rmse', 1202.7976338659253

Params: bagging_fraction=0.8501592630427213 bagging_freq=2.0 feature_fraction=0.7621275783003609 lambda_l1=6.227887150714633 lambda_l2=8.673183343728075 learning_rate=0.006560441766214547 max_bin=159.0 min_data_in_leaf=18.0 min_sum_hessian_in_leaf=6.491788633470996 num_leaves=45.0
nb_trees=7554 val_loss=OrderedDict([('rmse', 787.0727103301944)])                                                      
nb_trees=6382 val_loss=OrderedDict([('rmse', 798.4217539604331)])                                                      
nb_trees=5973 val_loss=OrderedDict([('rmse', 763.8465291855314)])                                                      
val_r2_score=0.8908334857226773                                                                                        
NEW BEST SCORE=0.8908334857226773                                                                                      
                                                                                                                      

nb_trees=9998 val_loss=OrderedDict([('rmse', 789.1973288922426)])                                                      
nb_trees=9994 val_loss=OrderedDict([('rmse', 799.979134193469)])                                                       
nb_trees=9936 val_loss=OrderedDict([('rmse', 761.039475435889)])                                                       
val_r2_score=0.8903862576626674                                                                                        
                                                                                                                       
LightGBM objective call #157 cur_best_score=0.89103
Params: bagging_fraction=0.9416867771734293 bagging_freq=1.0 feature_fraction=0.7503373783867197 lambda_l1=9.814237050342033 lambda_l2=9.81863359217589 learning_rate=0.0055335008338507595 max_bin=180.0 min_data_in_leaf=20.0 min_sum_hessian_in_leaf=9.791095650645548 num_leaves=37.0
nb_trees=9892 val_loss=OrderedDict([('rmse', 785.9818797561627)]) 

LightGBM objective call #165 cur_best_score=0.89104
Params: bagging_fraction=0.962663076927143 bagging_freq=1.0 feature_fraction=0.9491132707454397 lambda_l1=9.632225084537298 lambda_l2=9.044210742171902 learning_rate=0.005952391486964579 max_bin=180.0 min_data_in_leaf=42.0 min_sum_hessian_in_leaf=8.681118657515949 num_leaves=17.0
nb_trees=10000 val_loss=OrderedDict([('rmse', 798.9163368710877)])                                                     
nb_trees=10000 val_loss=OrderedDict([('rmse', 807.9421893506309)])                                                     
nb_trees=10000 val_loss=OrderedDict([('rmse', 767.2726920075417)])                                                     
val_r2_score=0.8873720344288702                                                                                        
                                                                                                                       
LightGBM objective call #166 cur_best_score=0.89104
Params: bagging

KeyboardInterrupt: 